In [1]:
# Here are some libraries that we'll need for this process
import pandas as pd
from datetime import datetime
# import pyodbc as dbapi
# from tqdm import tqdm
# from IPython.core.display import Image
# from IPython import display
import requests
import os.path
from datetime import timedelta
import json
from collections import defaultdict
import re
import numpy as np
# import matplotlib.pyplot as plt
import csv
import requests
import pprint
import math
pp = pprint.PrettyPrinter(indent=4)
import snipeit
# from snipeit import Models
import random 
server='https://dev2.sawpit.app'
token=''


In [2]:
#load first sheet of excel into a dataframe
df = pd.read_excel('Master Inventory - RMJC 4-15-2023.xlsx', index_col=0, sheet_name=0) 

#show first row of xl spreadsheet
for index,row in df.iterrows():
    print(index,row)
    break


RMT 221001 0013 SRL# NEW                         NaN
Inventory                        RMT
Supplier                      Delson
PO#                              NaN
Order #                        12372
Date             2022-10-01 00:00:00
Type                    Heavy Timber
QTY                              1.0
Width                            6.0
Height                           6.0
Length                          10.0
BDF                             30.0
Species                   POC - FOHC
Grade              #2/Btr Appearance
Dryness                        Green
Dryness Today                    NaN
Summary Name     POC - FOHC - 6x6x10
Condition                        NaN
BDF Cost                       2.743
Freight Cost                    0.52
Landed Cost                    3.263
Value                          97.89
Markup                          1.25
Sell                        122.3625
Age                               10
Status                     Available
Date Used             

In [32]:
uniqueGrades = [x for x in df["Grade"].unique().tolist() if str(x) != 'nan']
uniqueDryness = [x for x in  df["Dryness"].unique().tolist() if str(x) != 'nan']
uniqueWidth = [x for x in df["Width"].unique().tolist() if str(x) != 'nan']
uniqueWidth.sort()
uniqueHeight = [x for x in df["Height"].unique().tolist() if str(x) != 'nan']
uniqueHeight.sort()
uniqueLength = [x for x in df["Length"].unique().tolist() if str(x) != 'nan']
uniqueLength.sort()
uniqueSpecies =  [x for x in df["Species"].unique().tolist() if str(x) != 'nan']
pp.pprint(uniqueGrades)
pp.pprint(uniqueDryness)
print(uniqueWidth)
print(uniqueHeight)
print(uniqueLength)
pp.pprint(uniqueSpecies)

['#2/Btr Appearance', '#2/Btr ', '#1', '#2']
['Green', 'Air Dried', 'Kiln Dried']
[2.0, 3.0, 3.5, 4.0, 5.0, 5.25, 5.5, 6.0, 7.0, 7.5, 7.75, 8.0, 11.0, 11.5, 12.0, 12.5]
[5.5, 6.0, 7.0, 7.5, 8.0, 9.0, 10.0, 11.0, 11.5, 11.75, 12.0, 12.5, 13.5, 15.0, 15.5, 16.0, 19.5]
[5.0, 6.0, 8.0, 10.0, 12.0, 14.0, 16.0, 18.0, 20.0, 22.0, 24.0, 28.0, 32.0]
['POC - FOHC', 'WRC - BHC', 'DF - FOHC']


In [18]:
# generate a list of beam sizes


#Standard-- Standard lengths of lumber
#shall be in multiples of 0.3048 m (1 foot) or 0.6096
#m (2 feet) as specified in the certified grading 

species_group = ["CEDAR","DOUGLAS FIR","PINE"]
thickness = [5,6,7,8,9,10,11,12,14,16]
width = [5,6,7,8,9,10,11,12,13,14,15,16,18,20]
length = [4,5,6,8,10,12,14,16,18,20,22,24,28,32]
pith = ["FOHC","HC","BHC"]


unique_summaries=[]

for sg in species_group:
    for t in thickness:
        for w in width:
            if w > t:
                for l in length:
                    for p in pith:
                        summary = f"{sg} - {p} - {t}x{w}x{l}"
                        unique_summaries.append(summary) 
    


print(len(unique_summaries))


def get_nominal_model_summary(species,pith,w,h,l):
    # get the PS20 standard lumber size from actual size
    # the model should represent what a supplier would call this line item 
    thickness = math.floor(w) 
    width = math.floor(h) 
    length = math.floor(l)
    summary = f"{species} - {pith} - {thickness}x{width}x{length}"
    return summary



10332


In [41]:
def summaryToModel(summ,useHC=True,useDims=True,useLength=True,useNominal=True):

   
    # s looks ike   "POC - FOHC - 6x6x10"
    #remove whitespace
    s = summ.split('-')
    species = s[0].strip()
    heartCenter = s[1].strip()
    dims = ''
    w=None
    h=None
    l=None
    bdf=0
    if len(s) > 2:
        dims = s[2].strip()

    if len(dims) > 2:
        #print(summ)
        dims = dims.split('x')
        w=float(dims[0])
        h=float(dims[1])
        
        if useNominal and w:
            w = math.floor(w)
        if useNominal and h:
            h = math.floor(h)

        if useLength:
            l=float(dims[2])
            if useNominal and l:
                l = math.floor(l)
            dims = 'x'.join([str(w),str(h),str(l)])
        else:
            dims = 'x'.join([str(w),str(h)])
            l=None

    if(w and h and l):
        bdf = math.floor(((w*h)/12)*l)

    # category 2 is heavy timber
    manufacturer_id = 1 # 1 is Unknown
    if species ==  "DF":
        manufacturer_id = 2
    if species ==  "POC":
        manufacturer_id = 3
    if species ==  "AYC":
        manufacturer_id = 4
    if species ==  "WRC":
        manufacturer_id = 5
    if species ==  "WO":
        manufacturer_id = 6

    # model_number can come from the supplier Invoice if neeeded
    model_number = ""
    
    summary = '-'.join([species])
    if useHC:
        summary = '-'.join([species,heartCenter])
    if useDims:
        summary = '-'.join([species,heartCenter,dims])
    # model dims are nominal,  asset dims are actual
    # we need to use length for model to rapidly enter a truck via a kit
    # for the same reason it may be faster to have FOHC in the model_no
    # 
    #can you please remove all the fields from this json array, except for db_column_name and default_value?

    defaults  = [   
        {   'db_column_name': '_snipeit_species_22',
            'default_value': species
        },
        {   'db_column_name': '_snipeit_pith_18',
            'default_value': heartCenter
        },
        {   'db_column_name': '_snipeit_thickness_4',
            'default_value': w
        },
        {   'db_column_name': '_snipeit_width_5',
            'default_value': h
        },
        {   'db_column_name': '_snipeit_length_7',
            'default_value': l
        },
        {   'db_column_name': '_snipeit_bdf_8',
            'default_value': bdf
        },
        {   'db_column_name': '_snipeit_pith_18',
            'default_value': heartCenter
        },
        {   'db_column_name': '_snipeit_grade_2',
            'default_value': "#1"
        },
        {   'db_column_name': '_snipeit_condition_9',
            'default_value': None
        },
        {   'db_column_name': '_snipeit_moisture_3',
            'default_value': 'S-GRN'
        },
        {   'db_column_name': '_snipeit_dryness_today_13',
            'default_value': None
        },
        {   'db_column_name': '_snipeit_bdf_cost_10',
            'default_value': None
        },
        {   'db_column_name': '_snipeit_freight_11',
            'default_value': None
        },
        {   'db_column_name': '_snipeit_markup_12',
            'default_value': 1.25
        }
    ]

    d = {"name":summary, "category_id":2, "manufacturer_id":manufacturer_id, "fieldset_id":2, "model_number":model_number, "default_fieldset_values":defaults}

    # assignCustomFieldsDefaultValues
    return d

def summaryToModelNoDims(s):
    return summaryToModel(s,useHC=True,useDims=False,useLength=False)

def summaryToModelNoLength(s):
    return summaryToModel(s,useHC=True,useDims=True,useLength=False)

def summaryToModelSpeciesOnly(s):
    return summaryToModel(s,useHC=False,useDims=False,useLength=False)

def modelToSummary(m,useHC=True):
    if useHC:
        return m['name']
    species = m['name'].split('-')[0].strip()
    dims = (m['name'].split('-')[1] or '').strip()
    return ' - '.join(species,m['model_number'],dims).strip()


In [46]:
#get a list of unique models from the spreadsheet
uniqueModels = list(map(summaryToModel, df["Summary Name"].unique().tolist()))
uniqueModels.extend( list(map(summaryToModelNoDims, df["Summary Name"].unique().tolist())))
uniqueModels.extend( list(map(summaryToModelSpeciesOnly, df["Summary Name"].unique().tolist())))
uniqueGrades = df["Grade"].unique().tolist()
uniqueDryness = df["Dryness"].unique().tolist()
uniqueModelsDict = {} #default of None
#make unique keys
for row in uniqueModels:
     uniqueModelsDict[row['name']]=row
uniqueModels = list(uniqueModelsDict.values())
print(len(uniqueModels))
pp.pprint(uniqueModels)

73
[   {   'category_id': 2,
        'default_fieldset_values': [   {   'db_column_name': '_snipeit_species_22',
                                           'default_value': 'POC'},
                                       {   'db_column_name': '_snipeit_pith_18',
                                           'default_value': 'FOHC'},
                                       {   'db_column_name': '_snipeit_thickness_4',
                                           'default_value': 6},
                                       {   'db_column_name': '_snipeit_width_5',
                                           'default_value': 6},
                                       {   'db_column_name': '_snipeit_length_7',
                                           'default_value': 10},
                                       {   'db_column_name': '_snipeit_bdf_8',
                                           'default_value': 30},
                                       {   'db_column_name': '_snipeit_pith_18',
   

In [55]:
#load a dict that maps name to model ids
M = snipeit.Models()
modelsDict = json.loads( M.get(server, token, 5000) )# Using default limit of 5000 for results
existing_model_ids = defaultdict() #default of None
for row in modelsDict['rows']:
     # csvSummary=(row)
     if row['name'] == 'DF-FOHC-8x8x24':
         pp.pprint(row)
     existing_model_ids[row['name']]=row['id']


def updateModel(modelID, payload):
    url = server + '/api/v1/models/' + str(modelID)
    headers = {
        "accept": "application/json",
        "Authorization": 'Bearer ' + token,
        "content-type": "application/json"
    }
    results = requests.patch(url, payload, headers=headers)
    return results.content

#upload the unique models to sawpit
uploadModels = True
if uploadModels:
    M = snipeit.Models()
    sumRegex = re.compile(r"(\w+) - (\w+) - ([0123456789.x]+)")
    newSummaryRegex = re.compile(r"(\w+)-([0123456789.x]+)")
    newSummaryRegex = re.compile(r"(\w+)")
    for m in uniqueModels:
        summaryMatch = newSummaryRegex.match(m['name'])
        if(summaryMatch is None):
            #invalid name, skip it
            continue
        existing_id = existing_model_ids.get(m['name'])
        if(existing_id is None):
            print(f"creating model {m['name']}")
            r = M.create(server, token,  json.dumps(m))
        else:
            print(f"updating model {m['name']}")
            r = updateModel(existing_id,m)
            #r = M.updateModel(server, token, str(existing_id),json.dumps(m))
        if("error" in r):
            print(r)


{   'assets_count': 0,
    'available_actions': {   'clone': True,
                             'delete': True,
                             'restore': False,
                             'update': True},
    'category': {'id': 2, 'name': 'Timber'},
    'created_at': {   'datetime': '2023-09-11 14:45:08',
                      'formatted': '2023-09-11 2:45PM'},
    'default_fieldset_values': [   {   'db_column_name': '_snipeit_species_22',
                                       'default_value': 'DF',
                                       'format': 'ANY',
                                       'name': 'Species',
                                       'required': True},
                                   {   'db_column_name': '_snipeit_pith_18',
                                       'default_value': 'FOHC',
                                       'format': 'ANY',
                                       'name': 'Pith',
                                       'required': True},
            

TypeError: a bytes-like object is required, not 'str'

In [3]:

#get the lastest list of models from the server
M = snipeit.Models()
modelsDict = json.loads( M.get(server, token, 5000) )# Using default limit of 5000 for results
existing_model_ids = defaultdict() #default of None
for row in modelsDict['rows']:
     # csvSummary=(row)
     existing_model_ids[row['name']]=row['id']


pp.pprint(modelsDict['rows'][1])
modelsFromServer = modelsDict['rows']
modelsFromServer[0]

{   'assets_count': 4,
    'available_actions': {   'clone': True,
                             'delete': False,
                             'restore': False,
                             'update': True},
    'category': {'id': 2, 'name': 'Timber'},
    'created_at': {   'datetime': '2023-08-23 12:33:28',
                      'formatted': '2023-08-23 12:33PM'},
    'default_fieldset_values': [   {   'db_column_name': '_snipeit_species_22',
                                       'default_value': None,
                                       'format': 'ANY',
                                       'name': 'Species',
                                       'required': True},
                                   {   'db_column_name': '_snipeit_pith_18',
                                       'default_value': None,
                                       'format': 'ANY',
                                       'name': 'Pith',
                                       'required': True},
            

{'id': 210,
 'name': 'POC-6x6',
 'manufacturer': {'id': 3, 'name': 'Port Orford Cedar'},
 'image': None,
 'model_number': 'FOHC',
 'depreciation': None,
 'assets_count': 277,
 'category': {'id': 2, 'name': 'Timber'},
 'fieldset': {'id': 2, 'name': 'Heavy Timber'},
 'default_fieldset_values': [{'name': 'Species',
   'db_column_name': '_snipeit_species_22',
   'default_value': None,
   'format': 'ANY',
   'required': True},
  {'name': 'Pith',
   'db_column_name': '_snipeit_pith_18',
   'default_value': None,
   'format': 'ANY',
   'required': True},
  {'name': 'Thickness',
   'db_column_name': '_snipeit_thickness_4',
   'default_value': None,
   'format': 'NUMERIC',
   'required': True},
  {'name': 'Width',
   'db_column_name': '_snipeit_width_5',
   'default_value': None,
   'format': 'NUMERIC',
   'required': True},
  {'name': 'Length',
   'db_column_name': '_snipeit_length_7',
   'default_value': None,
   'format': 'NUMERIC',
   'required': True},
  {'name': 'Grade',
   'db_column_nam

In [4]:
#load a dict that maps company name to company id 
C = snipeit.Company()
companiesDict = json.loads( C.get(server, token) )# Using default limit of 5000 for results
# pp.pprint(r)

# pp.pprint(modelsDict['rows'][1])
company_ids = defaultdict() #default of None
for row in companiesDict['rows']:
    company_ids[row['name']]=row['id']

company_ids

defaultdict(None,
            {'Found, Missing Label': 8,
             'Drops &amp; Offcuts': 7,
             'Trillium Dell': 6,
             'TETW': 5,
             'COTF': 4,
             'Delson': 3,
             'RMJC': 2,
             'RMT': 1})

In [5]:
#load a dict that maps supplier name to supplier id 
# S = snipeit.Supplier()
# suppliersDict = json.loads( S.get(server, token) )# Using default limit of 5000 for results
# pp.pprint(r)

# pp.pprint(modelsDict['rows'][1])
supplier_ids = defaultdict(lambda: 8,{
    'Delson':1,
    'Bluelinx':2,
    'Rosboro':3,
    'Boulder Lumber':4,
    'Hull-Oakes':5,
    'Customer':6,
    'TETW':7,
    'Unknown':8,
}) 

supplier_ids['Delson']

1

In [14]:
#define functions needed for upload
def strip(s):
    if isinstance(s, str):
        s = s.strip()
        if '-' == s: # change dash into blank
            return None
        return s
    return s

def formatDate(s):
    if isinstance(s, str):
        m, d, y = s.strip().split('/')
        return y+'-'+m.rjust(2,'0')+'-'+d.rjust(2,'0')
    return s

def getDims(row):
    summary = row['Summary Name']
    s = summary.split('-')
    species = s[0].strip()
    boxedHeart = s[1].strip()
    dims = None
    if len(s) > 2:
        dims = s[2].strip()
    return dims

def standardizeDryness(row):
    # ['Green', 'Air Dried', 'Kiln Dried', nan]

    dr = row['Dryness']
    if "Green" in dr:
        dr = "S-GRN"
    if "Air Dried" in dr:
        dr = "S-DRY"
    if "Kiln Dried" in dr:
        dr = "KD"
    return dr


def standardizeGrade(row):
    #  ['#2/Btr Appearance', '#2/Btr ', '#1', '#2', nan]
    gr = row['Grade']
    if "#2/Btr Appearance" in gr:
        gr = "#2/Btr App"
    return gr

def standardizeSpecies(row):
    # ['POC', 'AYC', 'DF', nan]
    summary = row['Summary Name']
    s = summary.split('-')
    species = s[0].strip()
    return species

def standardizePith(row):
    # ['FOHC', 'BHC', 'SC', '']
    summary = row['Summary Name']
    s = summary.split('-')
    boxedHeart = s[1].strip()
    return boxedHeart

def getModelId(row):
    #  POC-8x10
    summary = row['Summary Name']
    s = summary.split('-')
    species = s[0].strip()
    boxedHeart = s[1].strip()
    dims = ''
    if len(s) > 2:
        dims = s[2].strip()
        dims = dims.split('x')
        dims = 'x'.join([dims[0],dims[1],dims[2]]) 

    key = f'{species}-{boxedHeart}-{dims}'
    id = 0
    for m in modelsFromServer:
        if m['name'] == key:
            id = m['id']
    print("model id,key =",id,key)
    return id

def getRandGPS(row):
    topleft=[39.99103771661651, -105.07273998372258]
    #bottom right is black square se corner of property
    bottomright=[39.98901792900641, -105.07181577814869] 

    randLat = round(random.uniform(bottomright[0],topleft[0]),5)
    randLng = round(random.uniform(topleft[1],bottomright[1]),5)

    str = f'@{randLat},{randLng}'

    return str

def csvToAsset(row):
    summary = row['Summary Name']
    status_id = 10 #10 is imported, but does not have new label yet
    if 'Used' in row['Status']:
        status_id = 9
   
    # print("supplier ",strip(row['Supplier']),supplier_ids[strip(row['Supplier'])])
    d = {"asset_tag":row['SRL_OLD'], 
        "name":getDims(row), #We set the name to just the dims to help with sorting 
        "serial":row['SRL_OLD'], 
        "status_id":status_id, 
        "model_id": getModelId(row), 
        "company_id": company_ids[strip(row['Inventory'])], 
        "supplier_id": supplier_ids[strip(row['Supplier'])], 
        "location_id": 1, # 1 is unknown
        "archived": False,
        "warranty_months": None,
        "depreciate": False,
        "requestable": True,
        "rtd_location_id": None,
        "_snipeit_bdf_8": row['BDF'],
        "_snipeit_bdf_cost_10": row['BDF Cost'],
        "_snipeit_species_22": standardizeSpecies(row),
        "_snipeit_pith_18": standardizePith(row),
        "_snipeit_condition_9": row['Condition'],
        "_snipeit_date_used_14": row['Date Used'],
        "_snipeit_moisture_3": standardizeDryness(row),
        "_snipeit_dryness_today_13": row['Dryness Today'],
        "_snipeit_freight_11": row['Freight Cost'],
        "_snipeit_grade_2": standardizeGrade(row),
        "_snipeit_width_5": row['Height'],
        "_snipeit_length_7": row['Length'],
        "_snipeit_markup_12": row['Markup'],
        "_snipeit_project_15": row['Project'],
        "_snipeit_thickness_4": row['Width'],
        "_snipeit_last_gps_16": getRandGPS(row),
        "order_number": row['Order'],
        "book_value": row['Sell'],
        "purchase_cost": row['Value'],
        "purchase_date": formatDate(row['Date']),
    }
    #strip whitespace from values
    cleanDict =  {k: strip(v) for k, v in d.items()}
    return cleanDict

def getID(tag):
    try:
        A = snipeit.Assets()
        return A.getID(server, token, tag)
    except:
        return None

def updateDevice(DeviceID, payload):
    url = server + '/api/v1/hardware/' + DeviceID
    headers = {
        "accept": "application/json",
        "Authorization": 'Bearer ' + token,
        "content-type": "application/json"
    }
    results = requests.patch(url, headers=headers, data=payload)
    return results.content
        
def getJson(tag):
    existing_id = getID(tag)
    A = snipeit.Assets()
    try:
        results = A.getDetailsByID(server, token, existing_id)
        # jsonData = json.loads((results).decode('utf-8').replace("'",'"'))
        pp.pprint(results)
    except:
         print(f'Exception geting json for  {tag} ')

#getJson('RMT 221001 0067')

In [16]:
#this cell will do the upload

def createOrUpdateAsset(asset,update=False):
    tag = asset['asset_tag']
    A = snipeit.Assets()
    existing_id = getID(tag)
    # pp.pprint(asset)
    try:
        if existing_id is None:
            r = A.create(server, token, json.dumps(asset))
            if 'error' in r:
                print(f'uploading {tag} status error')
                print(r)
                r = json.loads(r)
                print(f'{tag} error msg = ' + r["messages"])
            else:
                print(f'{tag} created ok')
        else:
            if(update):
                r = A.updateDevice(server, token, str(existing_id) , json.dumps(asset))
                if r:
                    print(f'{tag} updated ok')
            else:
                print(f'{tag} exists, skipping upload')
    except:
        print(f'Exception uploading {tag} ')

InventoryCSVfn = 'Master Inventory - RMJC 4-15-2023 - Inventory Master List.csv'

#TODO: strip whitespace from column names
skiprows = 0
rowLimit = 1000
with open(InventoryCSVfn, newline='') as csvfile:
    sumRegex = re.compile(r"(\w+) - (\w+) - ([0123456789.x]+)")
    reader = csv.DictReader(csvfile)
    rowcount = 0
    for row in reader:
        rowcount+=1
        if rowcount < skiprows:
            continue

        if(len(row['SRL_OLD']) < 3):
            print(f"Row {rowcount} is missing SRL_OLD, skipping it")
            continue
        summaryMatch = sumRegex.match(row['Summary Name'])
        if(summaryMatch is None):
            print(f"Row {rowcount} has bad summary {row['Summary Name']}, skipping it")
            continue
        asset = csvToAsset(row)
        if(asset['model_id']==[0]):
            print(f"Row {rowcount}  {row['Summary Name']} has bad model_id{row['model_id']}, skipping it")
            continue

        createOrUpdateAsset(asset,True)
        if rowcount > rowLimit:
            break

    


model id,key = 480 POC-FOHC-6x6x10
RMT 221001 0013 updated ok
model id,key = 480 POC-FOHC-6x6x10
RMT 221001 0014 updated ok
model id,key = 480 POC-FOHC-6x6x10
RMT 221001 0015 updated ok
model id,key = 480 POC-FOHC-6x6x10
RMT 221001 0016 updated ok
model id,key = 480 POC-FOHC-6x6x10
RMT 221001 0017 updated ok
model id,key = 481 POC-FOHC-6x6x8
RMT 221001 0018 updated ok
model id,key = 481 POC-FOHC-6x6x8
RMT 221001 0019 updated ok
model id,key = 481 POC-FOHC-6x6x8
RMT 221001 0020 updated ok
model id,key = 481 POC-FOHC-6x6x8
RMT 221001 0021 updated ok
model id,key = 481 POC-FOHC-6x6x8
RMT 221001 0022 updated ok
model id,key = 481 POC-FOHC-6x6x8
RMT 221001 0023 updated ok
model id,key = 481 POC-FOHC-6x6x8
RMT 221001 0024 updated ok
model id,key = 481 POC-FOHC-6x6x8
RMT 221001 0025 updated ok
model id,key = 481 POC-FOHC-6x6x8
RMT 221001 0026 updated ok
model id,key = 481 POC-FOHC-6x6x8
RMT 221001 0027 updated ok
model id,key = 481 POC-FOHC-6x6x8
RMT 221001 0028 updated ok
model id,key = 481 

In [ ]:
#this cell will upload fake maps and gps data

#root@limbertimber:/var/www/html/snipeit/public/uploads/assets# ls -la asset-image-1092-OYQSyNFi3U.jpg
#-rw-r--r-- 1 www-data www-data 94952 Aug 19 15:12 asset-image-1092-OYQSyNFi3U.jpg


# import random
# import requests

# def getRandMapImg():
#     mapGrids="B4,C1,C6,C7,C8,CA,CG,CH,D2,D3,D4,D5,E2,E3,E4,E5,E6,E9,EC,G5,G8,GA,GB,H1,H6,H7,H8,H9,HA,HB,HC,HD,HE,HF,HG,HL,HI,HH,HJ".split(',')
#     grid = random.choice(mapGrids)
#     fn = f"https://sawpit.app/uploads/maps/{grid}.jpg"
#     #pscp -r G:\work\snipe-it\rmjImport\maps root@sawpit.app:/var/www/html/snipeit/public/uploads
#     #chmod -R 644 /var/www/html/snipeit/public/uploads/maps
#     #chown -R www-data:www-data /var/www/html/snipeit/public/uploads/maps
#     return fn

# def setRandomMapImg(tag):
#     #TODO: set better gps cords
#     existing_id = getID(tag)
#     asset = {
#         'image':getRandMapImg(),
#         '_snipeit_lastgpsping_16':'@39.9898241,-105.0751819'
#         }
#     A = snipeit.Assets()
#     print(asset)
#     results = updateDevice(str(existing_id) , json.dumps(asset))
#     print(results)



# InventoryCSVfn = 'Master Inventory - RMJC 4-15-2023 - Inventory Master List.csv'

# skiprows = 0
# with open(InventoryCSVfn, newline='') as csvfile:
#     sumRegex = re.compile(r"(\w+) - (\w+) - ([0123456789.x]+)")
#     reader = csv.DictReader(csvfile)
#     rowcount = 0
#     for row in reader:
#         rowcount+=1
#         if rowcount < skiprows:
#             continue

#         if(len(row['SRL_OLD']) < 3):
#             print(f"Row {rowcount} is missing SRL_OLD, skipping it")
#             continue
#         summaryMatch = sumRegex.match(row['Summary Name'])
#         if(summaryMatch is None):
#             print(f"Row {rowcount} has bad summary {row['Summary Name']}, skipping it")
#             continue
#         setRandomMapImg(row['SRL_OLD'])
#         if rowcount > 10:
#             break


In [ ]:
#print first 5 assests
A = snipeit.Assets()
a = json.loads(A.get(server, token, 5)) # With a limit of results
pp.pprint(a['rows'][3])
